# 20151230-predict-household-income-from-census

Related post:  
https://stharrold.github.io/20151230-predict-household-income-from-census.html

Purpose: Predict total annual household income.

## Initialization

### Imports

In [1]:
cd ~

/home/samuel_harrold


In [3]:
# Import standard packages.
import collections
import json
import os
import pdb # TEST: Comment out pdb after testing.
import pprint
import sys
# Import installed packages.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Import local packages.
sys.path.insert(0, os.path.join(os.path.curdir, 'stharrold.github.io/content/static/dsdemos'))
# TEST: Comment out autoreload after testing.
%reload_ext autoreload
%autoreload 2
import dsdemos as dsd
# IPython magic.
%matplotlib inline

## Globals

In [4]:
path_static = os.path.join(os.path.expanduser(r'~'), r'stharrold.github.io/content/static')
basename = r'20151230-predict-household-income-from-census'
path_disk = os.path.abspath(r'/mnt/disk-20151227t211000z/')
path_acs = os.path.join(path_disk, r'www2-census-gov/programs-surveys/acs/')
path_hcsv = os.path.join(path_acs, r'data/pums/2013/5-Year/ss13hdc.csv') # 'hdc' = 'housing DC'
path_pcsv = os.path.join(path_acs, r'data/pums/2013/5-Year/ss13pdc.csv') # 'pdc' = 'person DC'
path_ddict = os.path.join(path_acs, r'tech_docs/pums/data_dict/PUMS_Data_Dictionary_2009-2013.txt')

## Extract-transform-load

**TODO:**
* Just use pandas. Acknowledge dask.

### CSVs

In [9]:
%%time
for path in [path_hcsv, path_pcsv]:
    with open(path) as fobj:
        nlines = sum(1 for _ in fobj)
    print("{path}:".format(path=path))
    print("    size (MB) = {size:.1f}".format(size=os.path.getsize(path)/1e6))
    print("    num lines = {nlines}".format(nlines=nlines))
print()
dfh = pd.read_csv(path_hcsv)
dfp = pd.read_csv(path_pcsv)
for (name, df) in [('dfh', dfh), ('dfp', dfp)]:
    print("{name} RAM usage (MB) = {mem:.1f}".format(
            name=name, mem=df.memory_usage().sum()/1e6))
print()

/mnt/disk-20151227t211000z/www2-census-gov/programs-surveys/acs/data/pums/2013/5-Year/ss13hdc.csv:
    size (MB) = 13.5
    num lines = 17501
/mnt/disk-20151227t211000z/www2-census-gov/programs-surveys/acs/data/pums/2013/5-Year/ss13pdc.csv:
    size (MB) = 30.5
    num lines = 30560

dfh RAM usage (MB) = 28.7
dfp RAM usage (MB) = 72.1

CPU times: user 1.56 s, sys: 80 ms, total: 1.64 s
Wall time: 1.71 s


/home/samuel_harrold/anaconda3/lib/python3.5/site-packages/IPython/core/magics/execution.py:2: DtypeWarning: Columns (107,108,109) have mixed types. Specify dtype option on import or set low_memory=False.
  """Implementation of execution-related magic functions."""


In [ ]:
percentiles = [0.1587, 0.5000, 0.8413] # +1 std. dev., mean/median, -1 std. dev. for normal dist.

In [11]:
dfh.head()

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
0,600,H,2009000000403,5,102,-9,3,11,1086032,1085467,...,6,25,30,32,26,6,36,6,18,19
1,NaN,H,2009000001113,5,103,-9,3,11,1086032,1085467,...,14,29,12,12,4,4,18,23,4,22
2,480,H,2009000001978,5,103,-9,3,11,1086032,1085467,...,65,12,14,37,36,41,57,36,11,34
3,NaN,H,2009000002250,5,105,-9,3,11,1086032,1085467,...,4,4,4,4,23,14,11,4,20,21
4,2500,H,2009000002985,5,101,-9,3,11,1086032,1085467,...,66,45,10,35,34,10,34,55,50,10


In [12]:
dfh.describe(percentiles=percentiles, include='all')

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
count,6561.000000,17500,1.750000e+04,17500,17500.000000,17500.000000,17500,17500,17500.000000,17500.000000,...,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000
unique,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,17500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,999.282731,NaN,2.011068e+12,5,56.427371,37.764171,3,11,1039364.231657,1048478.770229,...,17.050857,17.043486,17.050000,17.049029,17.048000,17.051543,17.053200,17.047029,17.046971,17.051486
std,1085.174484,NaN,1.401911e+09,0,55.291036,55.358495,0,0,31877.254257,29598.269890,...,17.593886,17.740566,17.534604,17.555515,17.558942,17.574232,17.623017,17.802284,17.267472,17.710924
min,0.000000,NaN,2.009000e+12,5,-9.000000,-9.000000,3,11,1000000.000000,1007549.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15.9%,200.000000,NaN,2.009001e+12,5,-9.000000,-9.000000,3,11,1000000.000000,1007549.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,790.000000,NaN,2.011001e+12,5,101.000000,-9.000000,3,11,1035725.000000,1054614.000000,...,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
84.1%,1500.000000,NaN,2.013000e+12,5,104.000000,104.000000,3,11,1086032.000000,1085467.000000,...,30.000000,30.000000,30.000000,31.000000,30.000000,30.000000,30.000000,30.000000,31.000000,30.000000


In [13]:
dfp.head()

,RT,SERIALNO,SPORDER,PUMA00,PUMA10,ST,ADJINC,PWGTP,AGEP,CIT,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2009000000403,1,102,-9,11,1085467,20,38,1,...,6,26,31,32,26,6,36,6,19,20
1,P,2009000001113,1,103,-9,11,1085467,13,78,1,...,13,30,12,13,4,4,18,24,4,21
2,P,2009000001113,2,103,-9,11,1085467,25,39,1,...,26,50,23,20,8,7,38,41,7,37
3,P,2009000001113,3,103,-9,11,1085467,17,8,1,...,15,32,17,15,6,4,26,32,5,30
4,P,2009000001978,1,103,-9,11,1085467,37,53,1,...,65,12,13,37,36,41,57,36,11,33


In [15]:
dfp.describe(percentiles=percentiles, include='all')

,RT,SERIALNO,SPORDER,PUMA00,PUMA10,ST,ADJINC,PWGTP,AGEP,CIT,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
count,30559,3.055900e+04,30559.000000,30559.000000,30559.000000,30559,30559.000000,30559.000000,30559.000000,30559.000000,...,30559.000000,30559.000000,30559.000000,30559.000000,30559.000000,30559.000000,30559.000000,30559.000000,30559.000000,30559.000000
unique,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,30559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.011081e+12,1.850584,55.840243,38.259923,11,1048186.138192,20.268039,38.728198,1.471252,...,20.268039,20.268039,20.268039,20.268039,20.268039,20.268039,20.268039,20.268039,20.268039,20.268039
std,NaN,1.407751e+09,1.235291,55.336541,55.395391,0,29716.696630,13.310075,21.780122,1.201267,...,17.948280,18.178196,17.677372,17.792287,17.774249,17.800602,17.930861,17.889701,17.399463,18.245862
min,NaN,2.009000e+12,1.000000,-9.000000,-9.000000,11,1007549.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15.9%,NaN,2.009001e+12,1.000000,-9.000000,-9.000000,11,1007549.000000,11.000000,18.000000,1.000000,...,5.000000,5.000000,6.000000,5.000000,5.000000,5.000000,5.000000,5.000000,6.000000,5.000000
50%,NaN,2.011001e+12,1.000000,101.000000,-9.000000,11,1054614.000000,16.000000,35.000000,1.000000,...,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
84.1%,NaN,2.013000e+12,3.000000,104.000000,104.000000,11,1085467.000000,31.000000,63.000000,1.000000,...,34.000000,34.000000,33.000000,34.000000,34.000000,33.000000,33.000000,33.000000,33.000000,34.000000


### Data dictionary

http://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2009-2013.txt

In [23]:
ddict = dsd.census.parse_pumsdatadict(path=path_ddict)

## Select features

**Notes:**
* Example consumer databases: http://www.consumerreports.org/cro/money/consumer-protection/big-brother-is-watching/overview/index.htm?rurl=http%3A%2F%2Fwww.consumerreports.org%2Fcro%2Fmoney%2Fconsumer-protection%2Fbig-brother-is-watching%2Foverview%2Findex.htm
* Random forests are scale invariant, so they can accommodate non-linear transformation.
* Cast all values to floats so that compatable with most algorithms and can use </> logic. Otherwise less informationally dense and may require deeper tree structure to find features.
* To "map to float" ('b' is N/A, mapped to 0; 1 is Yes; 2 is No; other values are special):  
```python
test = pd.DataFrame(data=[['b', 1.0], ['1', 1.0], ['2', 1.0], ['3', 1.1], ['4', 1.1]], columns=['COL', 'ADJ'])
tfmask = test['COL'].isin(['b'])
test.loc[tfmask, 'COL'] = 0.0
test['COL'] = test['COL'].astype(float)
print(test.dtypes)
test
```
* To "adjust for inflation":  
```python
test['ADJ'] *= 1e-6
tfmask = test['COL'] >= 3.0
test.loc[tfmask, 'COL'] *= test.loc[tfmask, 'ADJ']
test
```
* TODO: Remove vacant units ('NP') from data frame.
* TODO: Filter categorical variables from metadata (those without '..').

In [66]:
record_type = 'PERSON RECORD'
print_detail = False
for key in ddict['record_types'][record_type]:
    desc = ddict['record_types'][record_type][key]['description']
    if not (
        (key.startswith('F') and (desc.endswith(' flag') or desc.endswith(' edit')))
        or ('WGTP' in key and "Weight replicate" in desc)):
        if print_detail:
            print(key)
            pprint.pprint(ddict['record_types'][record_type][key])
        else:
            print("{key}: {desc}".format(key=key, desc=desc))

RT: Record Type
SERIALNO: Housing unit/GQ person serial number
SPORDER: Person number
PUMA00: Public use microdata area code (PUMA) based on Census 2000 definition for data collected prior to 2012. Use in combination with PUMA10.
PUMA10: Public use microdata area code (PUMA) based on 2010 Census definition for data Collected in 2012 or later. Use in combination with PUMA00.
ST: State Code
ADJINC: Adjustment factor for income and earnings dollar amounts (6 implied decimal places)
PWGTP: Person's weight
AGEP: Age
CIT: Citizenship status
CITWP05: Year of naturalization write-in for data collected prior to 2012
CITWP12: Year of naturalization write-in for data collected in 2012 or later
COW: Class of worker
DDRS: Self-care difficulty
DEAR: Hearing difficulty
DEYE: Vision difficulty
DOUT: Independent living difficulty
DPHY: Ambulatory difficulty
DRAT: Veteran service connected disability rating (percentage)
DRATX: Veteran service connected disability rating (checkbox)
DREM: Cognitive diffic

In [85]:
# Include columns that I think companies can easily get.
# for column details: https://www.census.gov/programs-surveys/acs/technical-documentation/pums/documentation.2013.html
# target: HINCP

cols_include = {
    'HOUSING RECORD': [
        'SERIALNO', 'PUMA00', 'PUMA10', 'ST', 'ADJHSG', 'ADJINC', 'WGTP', 'NP', 'BDSP', 'BLD', 'HINCP', 'R18', 'R65'],
    'PERSON RECORD': [
        'SERIALNO', 'SPORDER', 'PWGTP', 'AGEP', 'MAR', 'SCHL', 'INDP']
    }

In [86]:
for record_type in cols_include:
    print(record_type)
    for var_name in cols_include[record_type]:
        desc = ddict['record_types'][record_type][var_name]['description']
        print("{var}: {desc}".format(var=var_name, desc=desc))
    print()

HOUSING RECORD
SERIALNO: Housing unit/GQ person serial number
PUMA00: Public use microdata area code (PUMA) based on Census 2000 definition for data collected prior to 2012. Use in combination with PUMA10.
PUMA10: Public use microdata area code (PUMA) based on 2010 Census definition for data collected in 2012 or later. Use in combination with PUMA00.
ST: State Code
ADJHSG: Adjustment factor for housing dollar amounts (6 implied decimal places)
ADJINC: Adjustment factor for income and earnings dollar amounts (6 implied decimal places)
WGTP: Housing Weight
NP: Number of person records following this housing record
BDSP: Number of bedrooms
BLD: Units in structure
HINCP: Household income (past 12 months)
R18: Presence of persons under 18 years in household (unweighted)
R65: Presence of persons 65 years and over in household (unweighted)

PERSON RECORD
SERIALNO: Housing unit/GQ person serial number
SPORDER: Person number
PWGTP: Person's weight
AGEP: Age
MAR: Marital status
SCHL: Educational

In [80]:
record_type = 'PERSON RECORD'
var_codes = ddict['record_types'][record_type]['INDP']['var_codes']
indp_abbr = dict()
for var_code in var_codes.keys():
    indp_abbr[var_code] = var_codes[var_code].split(sep='-', maxsplit=1)[0][:3]
print(sorted(set(indp_abbr.values())))

['ADM', 'AGR', 'CON', 'EDU', 'ENT', 'EXT', 'FIN', 'INF', 'MED', 'MFG', 'MIL', 'Not', 'PRF', 'RET', 'SCA', 'SRV', 'TRN', 'UNE', 'UTL', 'WHL']


Actions for included columns:  
* HOUSING RECORD
    * SERIALNO: Use to join to PERSON RECORD.
    * PUMA00, PUMA10, ST: Combine and lookup lat-lon coordinates from census.gov.
    * ADJHSG, ADJINC: Multiply against other columns to adjust for inflation. See https://www.census.gov/library/publications/2009/acs/pums.html App 5.
    ADJHSG: CONP, ELEP, FULP, GASP, GRNTP, INSP, MHP, MRGP, SMOCP, RNTP, SMP, WATP
    ADJINC: INTP, OIP, PAP, PERNP, PINCP, RETP, SEMP, SSIP, SSP, WAGP
    * WGTP: Confirm with user verification file.
    * NP: Numerical. Map to float.
    * BDSP: Numerical. Map to float.
    * BLD: Map to median income.
    * HINCP: Include. Map household income to float. Adjust all with ADJINC.
    * R18: Include. Presence of persons under 18 years in household. Map to float.
    * R65: Include. Presence of persons 60+ years in household.
* PERSON RECORD
    * SERIALNO, SPORDER: Use as index.
    * PWGTP: Confirm user verification file.
    * AGEP: Map to float.
    * MAR: Map to float.
    * SCHL: Map to float.
    * INDP: (1) Map to float. (2) Map categories to median income.

## Export ipynb to html

In [158]:
!date --rfc-3339='seconds'

2016-01-01 05:40:17+00:00


In [ ]:
path_ipynb = os.path.join(path_static, basename, basename+'.ipynb')
for template in ['basic', 'full']:
    path_html = os.path.splitext(path_ipynb)[0]+'-'+template+'.html'
    cmd = ['jupyter', 'nbconvert', '--to', 'html', '--template', template, path_ipynb, '--output', path_html]
    print(' '.join(cmd))
    subprocess.run(args=cmd, check=True)